In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:1]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/casia-dataset/casia/CASIA2/Au/Au_pla_30322.jpg
/kaggle/input/casia-dataset/casia/CASIA2/Tp/Tp_D_NRN_M_N_nat00025_nat00028_11037.jpg
/kaggle/input/casia-dataset/casia/CASIA1/check.py
/kaggle/input/casia-dataset/casia/CASIA1/Sp/Sp_D_NNN_A_txt0065_txt0067_0151.jpg
/kaggle/input/casia-dataset/casia/CASIA1/Au/Au_art_0084.jpg
/kaggle/input/casia-dataset/CASIA2/Au/Au_pla_30322.jpg
/kaggle/input/casia-dataset/CASIA2/Tp/Tp_D_NRN_M_N_nat00025_nat00028_11037.jpg
/kaggle/input/casia-dataset/CASIA1/check.py
/kaggle/input/casia-dataset/CASIA1/Sp/Sp_D_NNN_A_txt0065_txt0067_0151.jpg
/kaggle/input/casia-dataset/CASIA1/Au/Au_art_0084.jpg


In [2]:
au_images = []
tp_images = []
for dirname, _, filenames in os.walk('/kaggle/input/casia-dataset/CASIA2'):
    for filename in filenames:
        img_path = os.path.join(dirname, filename)
        if 'Thumbs.db' in img_path:
            continue  # Skip 'Thumbs.db' file
        if 'Au' in img_path:
            au_images.append(img_path)
        else:
            tp_images.append(img_path)

In [3]:
print(len(au_images)) 
print(len(tp_images))

7491
5123


In [4]:
import matplotlib.pyplot as plt

In [ ]:
fig, axs = plt.subplots(nrows=8 ,ncols=5, figsize=(16,16))
    
for ax,img in zip(axs.flat, au_images[:40]):
    try:
        im = plt.imread(img)
        ax.imshow(im)
        ax.set_xticks([])
        ax.set_yticks([])
    except:
        pass
plt.show()

In [ ]:
fig, axs = plt.subplots(nrows=8 ,ncols=5, figsize=(16,16))
    
for ax,img in zip(axs.flat, tp_images[:40]):
    try:
        im = plt.imread(img)
        ax.imshow(im)
        ax.set_xticks([])
        ax.set_yticks([])
    except:
        pass
plt.show()

In [7]:
import cv2
import numpy as np

In [8]:
def generate_ela_batch(image_paths, resave_quality=60, temp_dir="."):
    ela_images = []

    for image_path in image_paths:
        # Load the original image
        original_image = cv2.imread(image_path)

        # Convert original image to grayscale
        original_image_gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

        # Resave the image with a specified quality
        temp_path = os.path.join(temp_dir, "temp_resaved.jpg")
        cv2.imwrite(temp_path, original_image, [cv2.IMWRITE_JPEG_QUALITY, resave_quality])

        # Load the resaved image and convert to grayscale
        resaved_image = cv2.imread(temp_path, cv2.IMREAD_GRAYSCALE)

        # Calculate the pixel-wise differences between the two images
        ela_image = cv2.absdiff(original_image_gray, resaved_image)

        # Apply a blur to highlight differences
        ela_image = cv2.GaussianBlur(ela_image, (5, 5), 0)

        # Calculate the ELA scale
        ela_scale = np.abs(original_image_gray - ela_image)

        # Normalize the ELA scale to 0-255 range
        ela_scale = ((ela_scale - np.min(ela_scale)) / (np.max(ela_scale) - np.min(ela_scale))) * 255

        # Convert to uint8
        ela_scale = ela_scale.astype(np.uint8)

        # Cleanup: Delete temporary resaved image
        os.remove(temp_path)

        ela_images.append(ela_scale)

    return ela_images

ela_au_images = generate_ela_batch(au_images)
ela_tp_images = generate_ela_batch(tp_images)


In [ ]:
print(len(ela_au_images))
print(ela_au_images)

In [ ]:
print(len(ela_tp_images))
print(ela_tp_images)

In [11]:
import cv2
import numpy as np

resized_au_images = []
for au_image, ela_au_image in zip(au_images, ela_au_images):
    if isinstance(au_image, str):
        au_image = cv2.imread(au_image)

    # Checking if ela_au_image has the expected shape
    if len(ela_au_image.shape) < 2:
        continue 

    desired_shape = ela_au_image.shape[:2][::-1]# (height,width) --> (width,height)
    resized_au = cv2.resize(au_image, desired_shape)

    resized_au_images.append(resized_au)

# print(resized_au_images)
print(len(resized_au_images))

7491


In [12]:
import cv2
import numpy as np

resized_tp_images = []

for tp_image, ela_tp_image in zip(tp_images, ela_tp_images):    
    if isinstance(tp_image, str):
        tp_image = cv2.imread(tp_image)

    if len(ela_tp_image.shape) < 2:
        continue 

    desired_shape = ela_tp_image.shape[:2][::-1]
    resized_tp = cv2.resize(tp_image, desired_shape)

    resized_tp_images.append(resized_tp)

# print(resized_tp_images)
print(len(resized_tp_images))


5123


 # **1. MSE**

In [109]:
import cv2
import numpy as np

class MSE:
    def __init__(self):
        pass

    def convert_to_gray(self, img):
        return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    def calculate_mse(self, img1, img2):
        img1_gray = self.convert_to_gray(img1)
        img2_gray = img2  # Assuming img2 is already in grayscale
        
        err = np.sum((img1_gray.astype("float") - img2_gray.astype("float")) ** 2)
        err = err // (img1_gray.shape[0] * img1_gray.shape[1])
        return err

# mse_calculator = MSE()
# mse = []
# for resized_au_image, ela_image in zip(resized_au_images[:1000], ela_au_images[:1000]):
#     mse_value = mse_calculator.calculate_mse(resized_au_image, ela_image)
#     mse.append(mse_value)

# print(len(mse))
# print(mse)


1000
[9.0, 61.0, 148.0, 38.0, 201.0, 1.0, 2.0, 14.0, 360.0, 8.0, 158.0, 540.0, 37.0, 33.0, 164.0, 4.0, 1.0, 74.0, 24.0, 7.0, 13.0, 1.0, 26.0, 4.0, 0.0, 61.0, 22.0, 33.0, 2.0, 1.0, 8.0, 3.0, 6.0, 127.0, 25.0, 9.0, 4.0, 8.0, 46.0, 8.0, 1.0, 306.0, 0.0, 12.0, 37.0, 1.0, 9.0, 67.0, 6.0, 5.0, 2.0, 0.0, 56.0, 11.0, 60.0, 23.0, 27.0, 47.0, 56.0, 69.0, 79.0, 108.0, 9.0, 19.0, 30.0, 3.0, 129.0, 2.0, 6.0, 14.0, 52.0, 159.0, 1.0, 0.0, 3.0, 3.0, 9.0, 7.0, 0.0, 355.0, 107.0, 5.0, 10.0, 0.0, 13.0, 0.0, 21.0, 244.0, 2.0, 3.0, 209.0, 109.0, 2.0, 306.0, 84.0, 16.0, 9.0, 0.0, 12.0, 3.0, 150.0, 0.0, 436.0, 36.0, 13.0, 160.0, 176.0, 51.0, 17.0, 50.0, 10.0, 5.0, 1.0, 3.0, 1696.0, 4.0, 21.0, 2.0, 79.0, 39.0, 20.0, 1.0, 11.0, 2.0, 82.0, 0.0, 78.0, 89.0, 3.0, 26.0, 6.0, 6.0, 18.0, 160.0, 31.0, 0.0, 66.0, 180.0, 132.0, 38.0, 64.0, 4.0, 25.0, 1.0, 62.0, 196.0, 0.0, 56.0, 78.0, 9.0, 8.0, 2142.0, 24.0, 14.0, 1352.0, 26.0, 186.0, 4.0, 84.0, 7.0, 5.0, 84.0, 124.0, 21.0, 10.0, 102.0, 36.0, 164.0, 24.0, 23.0, 555.0, 

 # **2. MSE_LAB**

In [176]:
import cv2
import numpy as np

class MSE_LAB:
    def __init__(self):
        pass
    
    def convert_to_gray(self, img):
        return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    def calculate_mse(self, img1, img2):
        img1_gray = self.convert_to_gray(img1)
        
        mse = np.mean((img1_gray - img2)**2)
        return mse

# mse_calculator = MSE_LAB()
# mse = []

# for resized_au_image, ela_image in zip(resized_au_images[:100], ela_au_images[:100]):
#     mse_value = mse_calculator.calculate_mse(resized_au_image, ela_image)
#     mse.append(mse_value)
    
# print("Number of MSE values:", len(mse))
# print("MSE values:", mse)


 # **3. MSE OF MAGNITUDE**

In [177]:
import numpy as np
import cv2

class MSE_A:
    def __init__(self):
        pass

    def convert_to_gray(self, img):
        return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    def mse_a(self, au_image, ela_image):
        au_gray = self.convert_to_gray(au_image)
        ela_image_float = ela_image.astype(np.float32)

        # Calculate the squared difference of magnitudes
        diff = np.abs(np.abs(au_gray) - np.abs(ela_image_float))**2

        # Calculate the mean squared error
        mse = np.mean(diff)

        return mse

# mse_calculator = MSE_A()
# mse_a_values = []

# for au_image, ela_image in zip(resized_au_images, ela_au_images):
#     mse_a_value = mse_calculator.mse_a(au_image, ela_image)
#     mse_a_values.append(mse_a_value)

# print("Number of MSE of Magnitude values:", len(mse_a_values))
# print("MSE of Magnitude values:", mse_a_values)


# **4. MSE OF PHASE**

In [187]:
import cv2
import numpy as np

class MSE_P:
    def __init__(self):
        pass
    
    def preprocess_images(self, img1, img2):
        img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        return img1_gray, img2

    def mse_p(self, img1, img2):
        img1_gray, img2 = self.preprocess_images(img1, img2)

        # Convert images to complex numbers by applying FFT
        img1_fft = np.fft.fft2(img1_gray)
        img2_fft = np.fft.fft2(img2)

        # Calculate the difference of phases
        diff = np.angle(img1_fft) - np.angle(img2_fft)

        # Calculate the mean squared error
        mse = np.mean(diff**2)

        return mse

mse_phase_calculator = MSE_P()
mse_p_values = []

for au_image, ela_image in zip(resized_au_images[:100], ela_au_images[:100]):
    mse_a_value = mse_phase_calculator.mse_p(au_image, ela_image)
    mse_p_values.append(mse_a_value)

print("Number of mse phase:", len(mse_p_values))
print("MSE of phase values:", mse_p_values)


Number of mse phase: 100
MSE of phase values: [0.5363171641063248, 3.270666855037701, 0.48908644999492035, 1.750022900115974, 3.955626038992522, 0.07540761577218072, 0.07745923026514111, 1.821740882050072, 0.17843688264240865, 1.662332392918034, 2.8804336049627577, 4.6850774765937855, 3.6559767838035486, 0.20428535828932157, 2.9750715493494266, 0.9339216718507534, 0.6854890355168868, 0.3394830833055898, 2.153939610968099, 0.11680450977874397, 0.13753052041870048, 0.9475909036116416, 2.320657687097255, 1.102462362296566, 0.5062952575963573, 2.655272665568932, 2.337012297338969, 1.8122134427534713, 0.947635478394805, 0.7011234344530788, 2.8934669704210774, 0.9614150669142579, 1.414142229245889, 0.6916910820759171, 0.22663565038819214, 0.2469023514913605, 0.31545686437832887, 1.2687516302729682, 0.26589092687430177, 1.2528202757523486, 1.3061689545206574, 0.6163189988759212, 0.9022716204814554, 2.3922199319120194, 3.35065285498843, 0.35079292386680533, 2.069094600384337, 2.094822971153317

# **MAGNITUDE DISTORTION MEASURES**

In [131]:
import cv2
import numpy as np

class Median_M:
    # class Variable
    block_size = 8

    def __init__(self, au_image, ela_image):
        self.au_image = au_image
        self.ela_image = ela_image
        self.au_gray = self.convert_to_gray(au_image)

    def convert_to_gray(self, img):
        return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    def preprocess_images(self, img1, img2):
        max_dim = max(img1.shape[0], img2.shape[0], img1.shape[1], img2.shape[1])
        resized_img1 = cv2.resize(img1, (max_dim, max_dim))
        resized_img2 = cv2.resize(img2, (max_dim, max_dim))
        return resized_img1, resized_img2

    def calculate_Jm(self):
        self.au_gray, self.ela_image = self.preprocess_images(self.au_gray, self.ela_image)
        
        height, width = self.au_gray.shape[:2]
        num_blocks_height = height // Median_M.block_size
        num_blocks_width = width // Median_M.block_size
        Jm_values = []

        for i in range(num_blocks_height):
            for j in range(num_blocks_width):
                start_row = i * Median_M.block_size
                end_row = start_row + Median_M.block_size
                start_col = j * Median_M.block_size
                end_col = start_col + Median_M.block_size

                original_block = self.au_gray[start_row:end_row, start_col:end_col]
                modified_block = self.ela_image[start_row:end_row, start_col:end_col]

                distortion = np.abs(modified_block - original_block)
                median_distortion = np.median(distortion)
                Jm_values.append(median_distortion)

        return np.median(Jm_values)

# mse_m_values = []

# for au_image, ela_image in zip(resized_au_images[:100], ela_au_images[:100]):
#     mse_m_calculator = Median_M(au_image, ela_image)
#     mse_m_value = mse_m_calculator.calculate_Jm()
#     mse_m_values.append(mse_m_value)

# print("Number of mse phase:", len(mse_m_values))
# print("MSE of phase values:", mse_m_values)


Number of mse phase: 100
MSE of phase values: [3.0, 0.0, 18.0, 254.0, 254.0, 1.0, 1.0, 254.0, 16.0, 0.0, 254.0, 254.0, 0.0, 3.0, 254.0, 255.0, 254.0, 10.0, 255.0, 3.0, 4.0, 0.0, 254.0, 255.0, 0.0, 254.0, 0.0, 254.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 251.0, 4.0, 3.0, 0.0, 6.0, 254.0, 0.0, 243.0, 0.0, 254.0, 0.0, 0.0, 0.0, 254.0, 254.0, 0.0, 255.0, 0.0, 251.0, 0.0, 5.0, 254.0, 4.0, 0.0, 254.0, 254.0, 254.0, 0.0, 0.0, 255.0, 0.0, 0.0, 254.0, 0.0, 255.0, 0.0, 254.0, 254.0, 1.0, 0.0, 254.0, 0.0, 2.0, 0.0, 0.0, 254.0, 255.0, 254.0, 0.0, 0.0, 254.0, 0.0, 254.0, 0.0, 0.0, 254.0, 254.0, 255.0, 0.0, 254.0, 255.0, 253.0, 254.0, 0.0, 3.0, 255.0]


# **6. Phase Distortion Measures**

In [129]:
import cv2
import numpy as np

class MEDIAN_P:
    # class Variable
    block_size = 8

    def __init__(self, au_image, ela_image):
        self.au_image = au_image
        self.ela_image = ela_image
        self.au_gray = self.convert_to_gray(au_image)

    def convert_to_gray(self, img):
        return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    def preprocess_images(self, img1, img2):
        max_dim = max(img1.shape[0], img2.shape[0], img1.shape[1], img2.shape[1])
        resized_img1 = cv2.resize(img1, (max_dim, max_dim))
        resized_img2 = cv2.resize(img2, (max_dim, max_dim))
        return resized_img1, resized_img2

    def calculate_Jphi(self):
        self.au_gray, self.ela_image = self.preprocess_images(self.au_gray, self.ela_image)
        
        height, width = self.au_gray.shape[:2]
        num_blocks_height = height // MEDIAN_P.block_size
        num_blocks_width = width // MEDIAN_P.block_size
        Jphi_values = []

        for i in range(num_blocks_height):
            for j in range(num_blocks_width):
                start_row = i * MEDIAN_P.block_size
                end_row = start_row + MEDIAN_P.block_size
                start_col = j * MEDIAN_P.block_size
                end_col = start_col + MEDIAN_P.block_size

                block_image = self.au_gray[start_row:end_row, start_col:end_col]
                block_modified_image = self.ela_image[start_row:end_row, start_col:end_col]

                distortion = np.angle(np.fft.fft2(block_modified_image) - np.fft.fft2(block_image))
                median_distortion = np.median(distortion)

                Jphi_values.append(median_distortion)

        return np.median(Jphi_values)

# mse_p_values = []

# for au_image, ela_image in zip(resized_au_images[:10], ela_au_images[:10]):
#     mse_p_calculator = MEDIAN_P(au_image, ela_image)
#     mse_p_value = mse_p_calculator.calculate_Jphi()
#     mse_p_values.append(mse_p_value)

# print("Number of mse phase:", len(mse_p_values))
# print("MSE of phase values:", mse_p_values)


Number of mse phase: 10
MSE of phase values: [0.0, 0.0, 0.0, 0.03942239336280135, 0.05069939064302052, 0.0, 0.0, 2.664535259100376e-15, 0.0, 0.0]


# **7. Human Visual System**

In [132]:
import cv2
import numpy as np

class HVS1:
    def __init__(self, img1, img2):
        self.img1, self.img2 = self.preprocessed_images(img1, img2)

    def preprocessed_images(self, img1, img2):
        # U: function to simulate HVS effect, takes an image array and returns a transformed array
        # grayscale: boolean flag to convert images to grayscale (default: False)

        # Convert images to grayscale if needed
        if len(img1.shape) == 3:  # Convert img1 to grayscale if it's not
            img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        else:
            img1_gray = img1

        img2_gray = img2  # img2 is already grayscale

        # Resize images to the maximum width and height
        max_dim = max(img1_gray.shape[0], img2_gray.shape[0], img1_gray.shape[1], img2_gray.shape[1])
        resized_img1 = cv2.resize(img1_gray, (max_dim, max_dim))
        resized_img2 = cv2.resize(img2_gray, (max_dim, max_dim))

        return resized_img1, resized_img2

    def U(self, x):
        # U function that applies contrast stretching and x is input image array
        min_val = np.min(x)
        max_val = np.max(x)
        
        # Check if the range of values is non-zero
        if max_val != min_val:
            stretched = (x - min_val) / (max_val - min_val)
        else:
            stretched = x

        return stretched

    def calculate_h1(self):
        K = len(self.img1)  # number of distorted images or channels
        N = self.img1.shape[0] * self.img1.shape[1]  # number of pixels in each image

        h1_sum = 0.0  # initialize sum of absolute differences

        for k in range(K):
            # preprocess original and distorted images with U function
            original_u = self.U(self.img1[k])
            distorted_u = self.U(self.img2[k])

            # calculate sum of absolute differences between U(Ck(i,j)) and U(Ĉk(i,j))
            diff_sum = np.sum(np.abs(original_u - distorted_u))

            # calculate average absolute difference and add to H1 sum
            h1_sum += diff_sum / (N * (np.sum(original_u) / N))

        # calculate final H1 value as average over K distorted images
        H1 = h1_sum / K

        return H1

# h1_values = []

# for au_image, ela_image in zip(resized_au_images[:100], ela_au_images[:100]):
#     hvs1_calculator = HVS1(au_image, ela_image)
#     h1_value = hvs1_calculator.calculate_h1()
#     h1_values.append(h1_value)

# print("Number of H1 values:", len(h1_values))
# print("H1 values:", h1_values)


Number of H1 values: 100
H1 values: [0.01055842619354621, 0.011834999002208385, 0.015646095955328395, 0.02204140117756949, 0.06170870938579782, 0.0038330340471777412, 0.002674638217947638, 0.025777623290968887, 0.007175231684767774, 0.0005621378696021885, 0.031690547689403485, 0.12280519926207062, 0.11114384583538406, 0.005656576131784856, 0.06002319634025902, 0.016236663785940693, 0.009216856356938655, 0.00867885653196464, 0.014695443128566833, 0.006127172001563339, 0.0037510298525497104, 0.008635443934217295, 0.0332825678405257, 0.008353882937660784, 0.0005119588404053925, 0.029644236632017193, 0.007668261071307701, 0.020183488136439404, 0.004040954340229337, 0.0006835634412893156, 0.006074027569993669, 0.003588413785527083, 0.005204866002277672, 0.02499384849816858, 0.013967838220518879, 0.014842246437841494, 0.011605364338564392, 0.002145228631657058, 0.004353814976531383, 0.025662681497342483, 0.001120313672071137, 0.006725499486740245, 0.0021172572531171444, 0.01814039519548395, 

# **8. HVS2**

In [135]:
import numpy as np
from PIL import Image
from skimage.transform import resize

class HVS2:
    def __init__(self, img1, img2):
        self.img1, self.img2 = self.preprocessed_images(img1, img2)

    def preprocessed_images(self, img1, img2):
        max_height = max(img1.shape[0], img2.shape[0])
        max_width = max(img1.shape[1], img2.shape[1])

        resized_img1 = resize(img1, (max_height, max_width))
        resized_img2 = resize(img2, (max_height, max_width))

        return resized_img1, resized_img2

    def U(self, image):
        gamma = 1.5  # Example gamma value

        transformed_image = np.power(image, gamma)
        return transformed_image

    def calculate_h2(self):
        U_distorted = self.U(self.img2)
        U_original = self.U(self.img1)

        # Ensure that both images have the same number of channels
        if len(U_distorted.shape) == 3 and len(U_original.shape) == 2:
            U_original = np.stack([U_original] * 3, axis=-1)
        elif len(U_original.shape) == 3 and len(U_distorted.shape) == 2:
            U_distorted = np.stack([U_distorted] * 3, axis=-1)

        squared_diff = np.square(U_distorted - U_original)
        sum_squared_diff = np.sum(squared_diff)

        squared_original = np.square(U_distorted)
        sum_squared_original = np.sum(squared_original)

        H2 = sum_squared_diff / sum_squared_original

        return H2

# h2_values = []

# for au_image, ela_image in zip(resized_tp_images[:100], ela_tp_images[:100]):
#     h2_calculator = HVS2(au_image, ela_image)
#     h2_value = h2_calculator.calculate_h2()
#     h2_values.append(h2_value)

# print("Number of H2 values:", len(h2_values))
# print("H2 values:", h2_values)


Number of H2 values: 100
H2 values: [0.16965118129262688, 0.00984870683953346, 0.3149393870619173, 0.014650746765270168, 0.295219024642212, 0.04165269029703032, 0.030580947551144336, 0.1751802464303916, 0.22809554022461642, 0.16376471780367402, 0.20038859254395652, 0.11703757675056464, 0.04390233919456971, 0.19840673722670923, 0.08761415657637879, 0.20447347806935975, 0.10220225665630514, 0.12929058147529246, 0.1043386987172199, 0.022287026312900974, 0.0234694460722726, 0.08876476662223788, 0.010061734987378465, 0.12607642657033466, 0.030216242940968592, 0.17659262318435417, 0.010769793037932502, 0.06031362737791847, 0.03770774458124024, 0.01617372514387231, 0.015735054119769724, 0.04482026961894777, 0.06259186758113862, 0.0052616215624607075, 0.33448821522017885, 0.03208152799947994, 0.013658743756538173, 0.0180564624488202, 0.03416436919522507, 0.06862880741096004, 0.035286910655957945, 0.024360175861357972, 0.05027413520264912, 0.014622912047069858, 0.0056690148960406835, 0.02550843

# **9. HVS3**

In [137]:
import numpy as np
from PIL import Image
from skimage.transform import resize

class HVS2:
    def __init__(self, img1, img2):
        self.img1, self.img2 = self.preprocessed_images(img1, img2)

    def preprocessed_images(self, img1, img2):
        max_height = max(img1.shape[0], img2.shape[0])
        max_width = max(img1.shape[1], img2.shape[1])

        resized_img1 = resize(img1, (max_height, max_width))
        resized_img2 = resize(img2, (max_height, max_width))

        return resized_img1, resized_img2

    def U(self, image):
        gamma = 1.5  # Example gamma value

        transformed_image = np.power(image, gamma)
        return transformed_image

    def calculate_h2(self):
        U_distorted = self.U(self.img2)
        U_original = self.U(self.img1)

        # Ensure that both images have the same number of channels
        if len(U_distorted.shape) == 3 and len(U_original.shape) == 2:
            U_original = np.stack([U_original] * 3, axis=-1)
        elif len(U_original.shape) == 3 and len(U_distorted.shape) == 2:
            U_distorted = np.stack([U_distorted] * 3, axis=-1)

        squared_diff = np.square(U_distorted - U_original)
        sum_squared_diff = np.sum(squared_diff)

        squared_original = np.square(U_distorted)
        sum_squared_original = np.sum(squared_original)

        H3 = sum_squared_diff / sum_squared_original

        return H3

# h2_values = []

# for au_image, ela_image in zip(resized_tp_images[:100], ela_tp_images[:100]):
#     h2_calculator = HVS2(au_image, ela_image)
#     h2_value = h2_calculator.calculate_h2()
#     h2_values.append(h2_value)

# print("Number of H2 values:", len(h2_values))
# print("H2 values:", h2_values)


Number of H3 values: 100
H3 values: [0.02508489072712418, 0.00981588924632353, 0.003843722563652326, 0.004130444495506536, 0.1284000970179738, 0.008752361621732025, 0.0035100400837418297, 0.11577132480596404, 0.032416689006331705, 0.008526810789419935, 0.0033799134677345126, 0.006815592447916667, 0.0022660977971435486, 0.0074600217864923745, 0.003117420470792484, 0.010418900633169935, 0.004060952180351307, 0.033788118232367575, 0.01022183287377451, 0.008062385620915031, 0.010975628495912469, 0.006729157441783895, 0.014748759219058888, 0.014551100388071893, 0.09706065857332517, 0.003776600158292483, 0.006297987188241799, 0.0020263671874999997, 0.007295496323529411, 0.009419280407475491, 0.0019441378300272558, 0.013802189542483656, 0.0003950929330065359, 0.007296533522263071, 0.009271280126633986, 0.004033586090686274, 0.01555223651960784, 0.012603130106417847, 0.00496826171875, 0.015452186734068625, 0.006458317376429736, 0.0043036566840277775, 0.0063637103490044675, 0.03798652202498357,

# **10. Structural Content between two images.**

In [141]:
from skimage.metrics import structural_similarity as ssim

class SSIM:
    def __init__(self, image1, image2):
        self.img1, self.img2 = image1, image2
        self.preprocess()

    def preprocess(self):
        # Convert images to grayscale if needed
        if len(self.img1.shape) == 3:  # Convert img1 to grayscale if it's not
            self.img1 = cv2.cvtColor(self.img1, cv2.COLOR_BGR2GRAY)

        if len(self.img2.shape) == 3:  # Convert img2 to grayscale if it's not
            self.img2 = cv2.cvtColor(self.img2, cv2.COLOR_BGR2GRAY)

        # Resize images to the minimum shape
        min_height = min(self.img1.shape[0], self.img2.shape[0])
        min_width = min(self.img1.shape[1], self.img2.shape[1])
        
        self.img1 = cv2.resize(self.img1, (min_width, min_height))
        self.img2 = cv2.resize(self.img2, (min_width, min_height))

    def calculate_ssim(self):
        # Compute the SSIM score between the images
        score = ssim(self.img1, self.img2, multichannel=False)  # Grayscale images, so multichannel=False
        return score

# ssim_values = []

# for au_image, ela_image in zip(resized_tp_images[:100], ela_tp_images[:100]):
#     ssim_calculator = SSIM(au_image, ela_image)
#     ssim_value = ssim_calculator.calculate_ssim()
#     ssim_values.append(ssim_value)

# print("Number of SSIM values:", len(ssim_values))
# print("SSIM values:", ssim_values)


Number of SSIM values: 100
SSIM values: [0.9120011375535302, 0.9785099893723094, 0.9984906007718253, 0.9991405546645731, 0.954447341386384, 0.9988004268909011, 0.987236516737726, 0.982589584124312, 0.9966455101843202, 0.996739194400556, 0.9996478324566824, 0.9949193073389007, 0.9982018560412407, 0.9913435335292954, 0.9867274338286838, 0.9876320725461643, 0.9954710663903567, 0.9924074534643971, 0.998488703949052, 0.9847138891182804, 0.9785178569371927, 0.9985806093801355, 0.9977953580400876, 0.9379693905508122, 0.9480569850621314, 0.9987648658616867, 0.9916991968216589, 0.9926870846177773, 0.9996236251969743, 0.9995834106946488, 0.9988822240094092, 0.9984033376615626, 0.999865606858716, 0.9786042263659996, 0.998818343831226, 0.9823576882486699, 0.980565137723641, 0.9927382682299715, 0.9993728405542016, 0.9992794922972961, 0.9617596558483732, 0.9996122294850741, 0.9971392128389825, 0.9951147930246851, 0.99931656388469, 0.9972630470842232, 0.9959057644678071, 0.9970417937229965, 0.9992180

# **11. Normalized Cross Correlation.**

In [ ]:
import cv2
import numpy as np

class NCC:
    def __init__(self, image1, image2):
        self.img1, self.img2 = image1, image2
        self.preprocess()

    def preprocess(self):
        # Convert images to grayscale if needed
        if len(self.img1.shape) == 3:  # Convert img1 to grayscale if it's not
            self.img1 = cv2.cvtColor(self.img1, cv2.COLOR_BGR2GRAY)

        if len(self.img2.shape) == 3:  # Convert img2 to grayscale if it's not
            self.img2 = cv2.cvtColor(self.img2, cv2.COLOR_BGR2GRAY)

        # Resize images to the same dimensions
        self.img1 = cv2.resize(self.img1, (self.img2.shape[1], self.img2.shape[0]), interpolation=cv2.INTER_LINEAR)

    def ncc(self):
        # Convert images to float32
        image1 = self.img1.astype(np.float32)
        image2 = self.img2.astype(np.float32)

        # Calculate mean and standard deviation of each image
        mean1 = np.mean(image1)
        mean2 = np.mean(image2)
        std1 = np.std(image1)
        std2 = np.std(image2)

        # Subtract mean and divide by standard deviation for each image
        image1_norm = (image1 - mean1) / std1
        image2_norm = (image2 - mean2) / std2

        # Calculate NCC
        ncc_value = np.sum(image1_norm * image2_norm) / (image1.size * std1 * std2)

        return ncc_value

# ncc_values = []
# for au_image, ela_image in zip(resized_au_images, ela_au_images):
#     ncc_calculator = NCC(au_image, ela_image)
#     ncc_value = ncc_calculator.ncc()
#     ncc_values.append(ncc_value)

# print("Number of NCC values:", len(ncc_values))
# print("NCC values:", ncc_values)


# **12. Image Fidelity between two images.**

In [ ]:
import cv2
from skimage.metrics import mean_squared_error, structural_similarity

class IF:
    def __init__(self, image1, image2):
        self.img1, self.img2 = self.preprocess_images(image1, image2)

    def preprocess_images(self, image1, image2):
        # Convert to grayscale if needed
        if len(image1.shape) == 3:
            image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
        else:
            image1_gray = image1

        # Resize images to have the same dimensions
        max_height = max(image1_gray.shape[0], image2.shape[0])
        max_width = max(image1_gray.shape[1], image2.shape[1])

        img1_resized = cv2.resize(image1_gray, (max_width, max_height))
        img2_resized = cv2.resize(image2, (max_width, max_height))

        return img1_resized, img2_resized

    def image_fidelity(self):
        mse = mean_squared_error(self.img1, self.img2)
        ssim = structural_similarity(self.img1, self.img2, win_size=7, multichannel=False)

        # Return the AVERAGE of MSE and SSIM values
        # Lower MSE and higher SSIM values generally indicate better image fidelity and a closer resemblance between the images.
        return (mse + ssim) / 2

# if_values = []

# for au_image, ela_image in zip(resized_au_images, ela_au_images):
#     if_calculator = IF(au_image, ela_image)
#     if_value = if_calculator.image_fidelity()
#     if_values.append(if_value)

# print("Number of IF values:", len(if_values))
# print("IF values (MSE, SSIM):", if_values)


# **13. similarity between two images using Histogram.**

In [ ]:
import cv2

class Histogram:
    def __init__(self, image1, image2):
        self.img1, self.img2 = self.preprocess_images(image1, image2)

    def preprocess_images(self, image1, image2):
        # Convert to grayscale if needed
        if len(image1.shape) == 3:
            image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
        else:
            image1_gray = image1

        image2_gray = image2  # ela_image is already grayscale

        return image1_gray, image2_gray

    def calculate_similarity(self):
        # Calculate histograms
        original_hist = cv2.calcHist([self.img1], [0], None, [256], [0, 256])
        tampered_hist = cv2.calcHist([self.img2], [0], None, [256], [0, 256])

        # Normalize histograms
        original_hist = cv2.normalize(original_hist, original_hist).flatten()
        tampered_hist = cv2.normalize(tampered_hist, tampered_hist).flatten()

        # Compare histograms using histogram intersection metric
        intersection = cv2.compareHist(original_hist, tampered_hist, cv2.HISTCMP_INTERSECT)

        return intersection

# histogram_values = []

# for au_image, ela_image in zip(resized_au_images, ela_au_images):
#     histogram_calculator = Histogram(au_image, ela_image)
#     histogram_value = histogram_calculator.calculate_similarity()
#     histogram_values.append(histogram_value)

# print("Number of Histogram Intersection values:", len(histogram_values))
# print("Histogram Intersection values:", histogram_values)


# **Generating Dataset: AU and its ELA**

In [ ]:
data_original=[]

for au_image,ela_image in zip(resized_au_images,ela_au_images):
    feature={}
    
    # 1.
    mse_calculator = MSE()
    mse_value = mse_calculator.calculate_mse(au_image, ela_image)
    feature['mse']=mse_value
    
    # 2.
    mselab_calculator = MSE_LAB()
    mse_value = mselab_calculator.calculate_mse(au_image, ela_image)
    feature['mse_lab']=mse_value
    
    # 3.
    mse_mag_calculator = MSE_A()
    mse_a_value = mse_mag_calculator.mse_a(au_image, ela_image)
    feature['mse_magnitude']=mse_a_value
    
    # 4.
    mse_phase_calculator = MSE_P()
    mse_p_value = mse_a_value = mse_phase_calculator.mse_p(au_image, ela_image)
    feature['mse_phase']=mse_p_value
    
    
    # 5.
    mse_m_calculator = Median_M(au_image, ela_image)
    med_m_value = mse_m_calculator.calculate_Jm()
    feature['med_magnitude']=med_m_value
    
    # 6.
    mse_p_calculator = MEDIAN_P(au_image, ela_image)
    med_p_value = mse_p_calculator.calculate_Jphi()
    feature['med_phase']=med_p_value
    
    # 7.
    hvs1_calculator = HVS1(au_image, ela_image)
    h1_value = hvs1_calculator.calculate_h1()
    feature['hvs1']=h1_value
    
    # 8.
    h2_calculator = HVS2(au_image, ela_image)
    h2_value = h2_calculator.calculate_h2()
    feature['hvs2']=h2_value
    
    # 9.
    h3_calculator = HVS3(au_image, ela_image)
    h3_value = h3_calculator.calculate_h3()
    feature['hvs3']=h3_value
    
    # 10.
    ssim_calculator = SSIM(au_image, ela_image)
    ssim_value = ssim_calculator.calculate_ssim()
    feature["ssim"]=ssim_value
    
    # 11.
    ncc_calculator = NCC(au_image, ela_image)
    ncc_value = ncc_calculator.ncc()
    feature['ncc']=ncc_value
    
    # 12.
    if_calculator = IF(au_image, ela_image)
    if_value = if_calculator.image_fidelity()
    feature['if_value']=if_value
    
    # 13.
    histogram_calculator = Histogram(au_image, ela_image)
    histogram_value = histogram_calculator.calculate_similarity()
    feature['histogram']=histogram_value
    
    # 14. Au- Authentic Unaltered
    feature['class']='Original'
    
    data_original.append(feature)
    
    
print(len(data_original))    
print(data_original)


# **Generating Dataset: TP and its ELA**

In [ ]:
data_tampered=[]

for au_image,ela_image in zip(resized_tp_images,ela_tp_images):
    feature={}
    
    # 1.
    mse_calculator = MSE()
    mse_value = mse_calculator.calculate_mse(au_image, ela_image)
    feature['mse']=mse_value
    
    # 2.
    mselab_calculator = MSE_LAB()
    mse_value = mselab_calculator.calculate_mse(au_image, ela_image)
    feature['mse_lab']=mse_value
    
    # 3.
    mse_mag_calculator = MSE_A()
    mse_a_value = mse_mag_calculator.mse_a(au_image, ela_image)
    feature['mse_magnitude']=mse_a_value
    
    # 4.
    mse_phase_calculator = MSE_P()
    mse_p_value = mse_a_value = mse_phase_calculator.mse_p(au_image, ela_image)
    feature['mse_phase']=mse_p_value
    
    
    # 5.
    mse_m_calculator = Median_M(au_image, ela_image)
    med_m_value = mse_m_calculator.calculate_Jm()
    feature['med_magnitude']=med_m_value
    
    # 6.
    mse_p_calculator = MEDIAN_P(au_image, ela_image)
    med_p_value = mse_p_calculator.calculate_Jphi()
    feature['med_phase']=med_p_value
    
    # 7.
    hvs1_calculator = HVS1(au_image, ela_image)
    h1_value = hvs1_calculator.calculate_h1()
    feature['hvs1']=h1_value
    
    # 8.
    h2_calculator = HVS2(au_image, ela_image)
    h2_value = h2_calculator.calculate_h2()
    feature['hvs2']=h2_value
    
    # 9.
    h3_calculator = HVS3(au_image, ela_image)
    h3_value = h3_calculator.calculate_h3()
    feature['hvs3']=h3_value
    
    # 10.
    ssim_calculator = SSIM(au_image, ela_image)
    ssim_value = ssim_calculator.calculate_ssim()
    feature["ssim"]=ssim_value
    
    # 11.
    ncc_calculator = NCC(au_image, ela_image)
    ncc_value = ncc_calculator.ncc()
    feature['ncc']=ncc_value
    
    # 12.
    if_calculator = IF(au_image, ela_image)
    if_value = if_calculator.image_fidelity()
    feature['if_value']=if_value
    
    # 13.
    histogram_calculator = Histogram(au_image, ela_image)
    histogram_value = histogram_calculator.calculate_similarity()
    feature['histogram']=histogram_value
    
    # 14. Au- Authentic Unaltered
    feature['class']='Tampered'
    
    data_tampered.append(feature)
    
    
print(len(data_tampered))    
print(data_tampered)

# **Creating a Dataframe**

In [ ]:
import pandas as pd
df1=pd.DataFrame(data_original)
df2=pd.DataFrame(data_tampered)

data=[df1,df2]
dataset=pd.concat(data)
print(df1)
print(df2)
print(dataset)